In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import datetime as dt

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score


In [2]:
df = pd.read_excel('Online Retail.xlsx')
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [3]:
def data_overview(df):
    print('Shape of Dataset'.center(120,'-'))
    print(f'number of Rows {df.shape[0]}')
    print(f'number of Columns {df.shape[1]}')
    print('Data'.center(120,'-'))
    print(df)
    print('Data Types'.center(120,'-'))
    print(df.dtypes.value_counts())
    print('Missing Values'.center(120,'-'))
    print(df.isnull().sum()[df.isnull().sum()>0].sort_values(ascending = False))
    print('Duplicated Values'.center(120,'-'))
    print(df.duplicated().sum())
    print(" Statistics of Data ".center(120,'-'))
    print(df.describe(include="all"))
    print("Data Info".center(120,'-'))
    print(df.info())
    
    
data_overview(df)

----------------------------------------------------Shape of Dataset----------------------------------------------------
number of Rows 541909
number of Columns 8
----------------------------------------------------------Data----------------------------------------------------------
       InvoiceNo StockCode                          Description  Quantity  \
0         536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1         536365     71053                  WHITE METAL LANTERN         6   
2         536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3         536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4         536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   
...          ...       ...                                  ...       ...   
541904    581587     22613          PACK OF 20 SPACEBOY NAPKINS        12   
541905    581587     22899         CHILDREN'S APRON DOLLY GIRL          6   
541906    581587     23

C:\Users\abdoe\AppData\Local\Temp\ipykernel_12180\3290990042.py:14: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  print(df.describe(include="all"))


        InvoiceNo StockCode                         Description  \
count    541909.0    541909                              540455   
unique    25900.0      4070                                4223   
top      573585.0    85123A  WHITE HANGING HEART T-LIGHT HOLDER   
freq       1114.0      2313                                2369   
first         NaN       NaN                                 NaN   
last          NaN       NaN                                 NaN   
mean          NaN       NaN                                 NaN   
std           NaN       NaN                                 NaN   
min           NaN       NaN                                 NaN   
25%           NaN       NaN                                 NaN   
50%           NaN       NaN                                 NaN   
75%           NaN       NaN                                 NaN   
max           NaN       NaN                                 NaN   

             Quantity          InvoiceDate      UnitPrice    

**Observations :**
- The Dataset has Rows: 541909 and Columns:8 
- The Dataset has 4 types of columns: strings(4), integer(1), float(2),datetime(1)
- The Dataset has Missing values in Customer ID (135080) and Description (1454)
- The Dataset has duplicates
- Invoice starts with the 'c' needs to be cleaned as it is cancelled transaction
- Also check for negative value and outliers in Quantity and Price

# Data Wrangling

In [4]:
df =df.dropna()
df.shape

(406829, 8)

In [5]:
#Remove Dublicated rows
df= df.drop_duplicates(keep="first")
df['Quantity'] = df['Quantity'].abs()

In [6]:
# converting negative values into postive values
df['UnitPrice'] = df['UnitPrice'].abs()

In [7]:
df['UnitPrice'].min()

0.0

In [8]:
df = df[df['UnitPrice'] > 0]


In [10]:
# Removing cancelled products from invoice
df  = df[~df.InvoiceNo.str.contains('C',na=False)]

In [11]:
# removing outliers from Quantity , Unitprice
for col in ['Quantity','UnitPrice']:
    low, high = df[col].quantile([0.05,0.95])
    df = df[df[col].between(low, high)]

In [12]:
# conert dtype [CustomerID] to str 
df['CustomerID'] = df['CustomerID'].astype(str)

# EDA : Feature Engineering

- **We are going to analysis the Customers based on below 3 factors:**
- **Recency**: Number of days since last purchase
- **Frequency**: Number of tracsactions
- **Monetary** : Total amount of transactions (revenue contributed)


In [13]:
df["Total_spend"] = df["Quantity"] * df["UnitPrice"]


In [12]:
print("Max date :",df["InvoiceDate"].max())
print("Min date :",df["InvoiceDate"].min())


Max date : 2011-12-09 12:50:00
Min date : 2010-12-01 08:26:00


In [22]:
Latest_Date = dt.datetime(2011,12,10)

RFM = df.groupby('CustomerID').agg({'InvoiceDate': lambda x: (Latest_Date - x.max()).days, 
                                                 'InvoiceNo': lambda x: x.nunique(), 
                                                 "Total_spend": lambda x: x.sum(),
                                                'Quantity':lambda x :x.sum(),
                                                'StockCode':lambda x: x.nunique(),
                                                'Description': lambda x:x
                                   })
                                
RFM['InvoiceDate'] = RFM['InvoiceDate'].astype(int)

#Renaming column names to Recency, Frequency and Monetary
RFM.rename(columns={'InvoiceDate': 'Recency', 
                         'InvoiceNo': 'Frequency', 
                         'Total_spend': 'Monetary' ,
                        'Quantity':'Total_Products_Purchased',
                       'StockCode' :'Unique_Products_Purchased',
                        'Description':'Popular_products'
                   }, inplace=True)

RFM.reset_index().head()


,CustomerID,Recency,Frequency,Monetary,Total_Products_Purchased,Unique_Products_Purchased,Popular_products
0,12347.0,2,7,3726.63,2091,98,WOODLAND DESIGN COTTON TOTE BAG
1,12348.0,248,3,90.20,140,6,SET OF 9 BLACK SKULL BALLOONS
2,12349.0,18,1,1287.15,619,66,ZINC FOLKART SLEIGH BELLS
3,12350.0,310,1,294.40,196,16,UNION JACK FLAG PASSPORT COVER
4,12352.0,36,7,1232.44,512,53,ZINC HEART FLOWER T-LIGHT HOLDER


In [20]:
RFM['Average Transaction Value'] = RFM['Monetary']/RFM['Frequency']

In [21]:
RFM.head()

,Recency,Frequency,Monetary,Total_Products_Purchased,Unique_Products_Purchased,Average Transaction Value
CustomerID,,,,,,
12347.0,2,7,3726.63,2091,98,532.375714
12348.0,248,3,90.20,140,6,30.066667
12349.0,18,1,1287.15,619,66,1287.150000
12350.0,310,1,294.40,196,16,294.400000
12352.0,36,7,1232.44,512,53,176.062857
